# Example problem: Monte-Carlo estimate of $\pi$

<img src="https://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif" width="25%" align=left alt="PI monte-carlo estimate"/>

## Problem description

Suppose we want to estimate the number $\pi$ using a [Monte-Carlo method](https://en.wikipedia.org/wiki/Pi#Monte_Carlo_methods), i.e. obtain a numerical estimate based on a random sampling approach, and that we want at least single precision floating point accuracy.

We take advantage of the fact that the area of a quarter circle with unit radius is $\pi/4$ and that hence the probability of a randomly chosen point inside a unit square to lie within that circle is $\pi/4$ as well.

So for N randomly chosen pairs $(x, y)$ with $x\in[0, 1)$ and $y\in[0, 1)$ we count the number $N_{circ}$ of pairs that also satisfy $(x^2 + y^2) < 1$ and estimage $\pi \approx 4 \cdot N_{circ} / N$.


## Monte-Carlo estimate with NumPy on a single CPU

* NumPy is the fundamental package for scientific computing with Python (https://numpy.org/).
* It contains a powerful n-dimensional array object and useful random number capabilities.

In [1]:
import numpy

In [2]:
def calculate_pi(size_in_bytes):
    
    """Calculate pi using a Monte Carlo method."""
    
    rand_array_shape = (int(size_in_bytes / 8 / 2), 2) # tuple
    
    # 2D random positions array
    xy = numpy.random.uniform(low=0.0, high=1.0, size=rand_array_shape)
    
    xy_inside_circle = (xy ** 2).sum(axis=1) < 1 # boolean

    pi = 4 * xy_inside_circle.sum() / xy_inside_circle.size

    print(f"\nfrom {xy.nbytes / 1e9} GB randomly chosen positions")
    print(f"   pi estimate: {pi}")
    print(f"   pi error: {abs(pi - numpy.pi)}\n")
    
    return pi

### Let's calculate...
Observe how the pi error decreases with an increasing number of randomly chosen positions!

In [3]:
%time pi = calculate_pi(size_in_bytes=10_000_000) # 10 MB
%time pi = calculate_pi(size_in_bytes=100_000_000) # 100 MB
%time pi = calculate_pi(size_in_bytes=1_000_000_000) # 1 GB


from 0.01 GB randomly chosen positions
   pi estimate: 3.1421312
   pi error: 0.0005385464102070081

CPU times: user 101 ms, sys: 49.9 ms, total: 150 ms
Wall time: 149 ms

from 0.1 GB randomly chosen positions
   pi estimate: 3.14295872
   pi error: 0.0013660664102070896

CPU times: user 1.06 s, sys: 510 ms, total: 1.57 s
Wall time: 1.57 s

from 1.0 GB randomly chosen positions
   pi estimate: 3.141846592
   pi error: 0.00025393841020671104

CPU times: user 8.1 s, sys: 4.99 s, total: 13.1 s
Wall time: 13.1 s


### Are we already better than single precision floating point resolution?

In [4]:
numpy.finfo(numpy.float32)

finfo(resolution=1e-06, min=-3.4028235e+38, max=3.4028235e+38, dtype=float32)

## We won't be able to scale the problem to several Gigabytes or Terabytes!

In [5]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000) # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.1416073984
   pi error: 1.4744810207023562e-05

CPU times: user 1min 3s, sys: 49.2 s, total: 1min 52s
Wall time: 1min 52s


In [6]:
# %time calculate_pi(size_in_bytes=100_000_000_000) # 100 GB
# %time calculate_pi(size_in_bytes=1_000_000_000_000) # 1 TB
# %time calculate_pi(size_in_bytes=10_000_000_000_000) # 10 TB

### Problems

* slowness of the numpy-only single CPU approach! (we could scale the problem using the [multiprocessing](https://docs.python.org/3.8/library/multiprocessing.html) and/or [threading](https://docs.python.org/3.8/library/threading.html) libraries)
* frontend/login node compute resources are shared and CPU, memory (and IO bandwidth) user demands will collide